In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as fn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger as TensorBoardLogger
import tensorboard


import numpy as np
from lstm_second import BasicLSTM



#### Let's test our model with a super simple validation from the StatQuest guy

In [ ]:
## create the training data for the neural network.
inputs = torch.tensor([[[0., 0.5, 0.25, 1.]], [[1., 0.5, 0.25, 1.]]])
labels = torch.tensor([0., 1.])

dataset = TensorDataset(inputs, labels) 
dataloader = DataLoader(dataset)

In [ ]:
mdl_simple = BasicLSTM(num_feat=1, num_hiddens=1, num_out=1, lr=0.01)


In [ ]:
print("Company A: Observed = 0, Predicted =", 
      mdl_simple(torch.tensor([[0., 0.5, 0.25, 1.]])).detach())
print("Company B: Observed = 1, Predicted =", 
      mdl_simple(torch.tensor([[1., 0.5, 0.25, 1.]])).detach())

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="simpleModel")

trainer_simple = pl.Trainer(max_epochs=1000,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
trainer_simple.fit(mdl_simple, train_dataloaders=dataloader)

In [ ]:
trainer_simple.test(mdl_simple,dataloaders=dataloader)

#### Now let's try to test our model with our stock data

In [ ]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, r'C:\Users\Spenc\Documents\GitHub\itcs-8156\utils')

from preprocessing import (market_prepro,
                           lstm_timeseries_feat_and_targ,
                           
)

In [ ]:
# st = "Stocks"
st = "ETFs"

#Input stock name
sn = "aadr" 
f = r'G:\My Drive\School\UNCC\P.h.D\Coursework\2023 - SPRING\ITCS 8156 - Machine Learning\Project\archive'
X_train, X_test, T_train, T_test = market_prepro(f,st,sn,False,splitdata=True, stdzr='minmax')

print(X_train.shape)
print(T_train.shape)
# X,T = market_prepro(f,st,sn,False,splitdata=False)

#number of days as features
day_feat = 2

#number of days to use as features
day_targ = 1
day_targ = day_targ - 1

# dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train[['Open','Low']], T_train, 4, 1,None)
# dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test[['Open','Low']], T_test, 4, 1,  None)

dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train, T_train, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])
dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test, T_test, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])

In [ ]:
mdl_stock = BasicLSTM(num_feat=7, num_hiddens=1, num_out=1, lr=0.01)


In [ ]:
mdl_stock.forward(ds_train[0][0])

In [ ]:
logger = TensorBoardLogger("lightning_logs", name="market")

trainer = pl.Trainer(max_epochs=10,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
trainer.fit(mdl_stock, train_dataloaders=dl_train)

In [ ]:
trainer.test(mdl_stock,dataloaders=dl_test)

#### Looking at the tensorboard test loss vs step it seems like most of the loss is coming from later in the predictions (farther from the training data.)

In [ ]:

def makepred(model, dataset):
    y = []
    t = []
    for ii in dataset:
        feat, lab = ii

        y.append(model.forward(feat).detach().numpy()[0])
        t.append(lab.numpy()[0])


    return y, t



In [ ]:
y_test, t_test = makepred(mdl_stock, ds_test)
y_train, t_train = makepred(mdl_stock, ds_train)

In [ ]:
from sklearn.metrics import r2_score



print('Training r2 ', r2_score(t_train, y_train) )
print('Testing r2 ', r2_score(t_test, y_test) )

In [ ]:
import matplotlib.pyplot as plt

def scatter_results(Y,T,title):
    plt.figure(figsize=(9,9))
    plt.scatter(Y,T)
    plt.xlabel('Model Prediction')
    plt.ylabel('True Value')
    plt.title(title)

In [ ]:
scatter_results(y_test, t_test, 'Testing')
scatter_results(y_train, t_train, 'Training')

In [ ]:
plt.plot(range(len(t_test)),t_test)
plt.plot(range(len(y_test)),y_test)

In [ ]:
actfn_labels = (['Sigmoid', 'ReLU', 'LeakyReLU', 'ELU', 'SELU', 'arcsinh', 'Swish', 'Softplus', 'Mish', 'Comb-H-Sine', 'tanh'])

results = []
for i in range(len(actfn_labels)):
    act1 = actfn_labels[i]

    for j in range(len(actfn_labels)):
        act2 = actfn_labels[j]
        display("Activation Function 1: ", act1)
        display("Activation Function 2: ", act2)
        mdl_stock = BasicLSTM(num_feat=7, num_hiddens=1, num_out=1, lr=0.01, actfn1=act1, actfn2=act2)
        mdl_stock.forward(ds_train[0][0])

        logger = TensorBoardLogger("lightning_logs", name="market")
        trainer = pl.Trainer(max_epochs=5,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
        trainer.fit(mdl_stock, train_dataloaders=dl_train)
        trainer.test(mdl_stock,dataloaders=dl_test)
        torch.save(mdl_stock, act1+act2)

        y_test, t_test = makepred(mdl_stock, ds_test)
        y_train, t_train = makepred(mdl_stock, ds_train)

        train_score = r2_score(t_train, y_train)
        test_score = r2_score(t_test, y_test)

        plt.plot(range(len(t_test)),t_test)
        plt.plot(range(len(y_test)),y_test)

        add_res = [act1, act2, train_score, test_score]

        results.append(add_res)

display("Activation Fn 1, Activation Fn 2, Training R2, Testing R2")
display(results)


In [ ]:
display("Activation Fn 1, Activation Fn 2, Training R2, Testing R2")
display(results)

In [ ]:
torch.save(results, 'ACT_FUNC_Results')

In [ ]:
import pandas as pd
import seaborn as sns
res = pd.DataFrame(torch.load('ACT_FUNC_Results'))
res.sort_values(3, ascending=True)
display(res.sort_values(3,ascending=False))


In [ ]:
actfn_labels = (['Sigmoid', 'ReLU', 'LeakyReLU', 'ELU', 'SELU', 'arcsinh', 'Swish', 'Softplus', 'Mish', 'Comb-H-Sine', 'tanh'])
train_map = pd.DataFrame(np.zeros([len(actfn_labels),len(actfn_labels)]))
train_map.columns = actfn_labels
train_map.index = actfn_labels

test_map = pd.DataFrame(np.zeros([len(actfn_labels),len(actfn_labels)]))
test_map.columns = actfn_labels
test_map.index = actfn_labels

for i in range(len(actfn_labels)):
    actfn1 = actfn_labels[i]

    for j in range(len(actfn_labels)):
        actfn2 = actfn_labels[j]
        train_map.iloc[i,j] = res.where((res[0]==actfn1)).where(res[1]==actfn2)[2].dropna()
        test_map.iloc[i,j] = res.where((res[0]==actfn1)).where(res[1]==actfn2)[3].dropna()




In [ ]:
sns.heatmap(train_map, vmin=0.8, vmax=1)

In [ ]:
sns.heatmap(test_map, vmin=0.95, vmax=1)

## Now that  we have determined the best activation functions, we can optimize the number of days we use as inputs.

We have done our initial testing predicting 1 day forward from the previous day, we will continue to predit 1 day forward, but we will include more input days.

In [ ]:
import sys
from sklearn.metrics import r2_score
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, r'C:\Users\Spenc\Documents\GitHub\itcs-8156\utils')

from preprocessing import (market_prepro,
                           lstm_timeseries_feat_and_targ,
                           
)


In [ ]:
# st = "Stocks"
st = "ETFs"

#Input stock name
sn = "aadr" 
f = r'G:\My Drive\School\UNCC\P.h.D\Coursework\2023 - SPRING\ITCS 8156 - Machine Learning\Project\archive'
X_train, X_test, T_train, T_test = market_prepro(f,st,sn,False,splitdata=True, stdzr='minmax')

print(X_train.shape)
print(T_train.shape)
# X,T = market_prepro(f,st,sn,False,splitdata=False)

actfn1 = ['Sigmoid']
actfn2 = ['tanh']

test_days = [2, 3, 5, 7, 10, 15, 20]

for i in range(len(actfn1)):

    act1fn1=actfn1[i]
    act2fn2=actfn2[i]

    for j in range(len(test_days)):
        #number of days as features
        day_feat = test_days[j]

        #number of days to use as features
        day_targ = 1
        day_targ = day_targ - 1

        # dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train[['Open','Low']], T_train, 4, 1,None)
        # dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test[['Open','Low']], T_test, 4, 1,  None)

        dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train, T_train, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])
        dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test, T_test, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])

        mdl_stock = BasicLSTM(num_feat=7, num_hiddens=1, num_out=1, lr=0.01, actfn1=act1fn1, actfn2=act2fn2)
        mdl_stock.forward(ds_train[0][0])

        logger = TensorBoardLogger("lightning_logs", name="market")
        trainer = pl.Trainer(max_epochs=5,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
        trainer.fit(mdl_stock, train_dataloaders=dl_train)
        trainer.test(mdl_stock,dataloaders=dl_test)
        torch.save(mdl_stock, act1fn1+act2fn2+str(day_feat))

        y_test, t_test = makepred(mdl_stock, ds_test)
        y_train, t_train = makepred(mdl_stock, ds_train)

        train_score = r2_score(t_train, y_train)
        test_score = r2_score(t_test, y_test)

        add_res_sweep = [act1fn1, act2fn2, day_feat, train_score, test_score]

        results_daysweep.append(add_res_sweep)


NOTE: LeakyReLU-ELU does not appear to work on higher number of input days. Max run was 15 days, blew up to epoch loss of inf the first time at 10 input days.

In [ ]:
torch.save(results_daysweep, 'Day Input Sweep Results')

In [ ]:
sweep = pd.DataFrame(results_daysweep)
display(sweep.sort_values(4, ascending=False))

In [ ]:
sweep_res=pd.DataFrame(results_daysweep)
train_sweep = sweep_res.where(sweep_res[0] == 'SELU').dropna()
plt.figure()
plt.suptitle('SELU - Sigmoid')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

In [ ]:
train_sweep = sweep_res.where(sweep_res[0] == 'ReLU').dropna()
plt.figure()
plt.suptitle('ReLU - Sigmoid')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

In [ ]:
train_sweep = sweep_res.where(sweep_res[0] == 'LeakyReLU').dropna()
plt.figure()
plt.suptitle('LeakyReLU - ELU')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([-1,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

In [ ]:
train_sweep = sweep_res.where(sweep_res[0] == 'Mish').dropna()
plt.figure()
plt.suptitle('Mish - Sigmoid')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

In [ ]:
train_sweep = sweep_res.where(sweep_res[0] == 'arcsinh').dropna()
plt.figure()
plt.suptitle('arcsinh - tanh')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

In [ ]:
train_sweep = sweep_res.where(sweep_res[0] == 'Sigmoid').dropna()
plt.figure()
plt.title('Sigmoid - tanh or BASE LSTM')
plt.subplot(121)
plt.plot(train_sweep[2], train_sweep[3])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Training')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

plt.subplot(122)
plt.plot(train_sweep[2], train_sweep[4])
plt.xlim([0,20])
plt.ylim([0.5,1])
plt.title('Testing')
plt.xlabel('# of input days')
plt.ylabel('r2 score')

# after considering the above plots, it appears that the best working model is the Mish-Sigmoid model with either 2,3, or 15 days of input data.

As such, we will train this model for 10 epochs and will attempt to use the same model on other stocks and determine the results.

In [ ]:
results_stock_sweep = []

# st = "Stocks"
st = "ETFs"

#Input stock name
stock_names = ["aadr", "aaxj", "acim", "actx", "acwf"]

actfn1 = ['Sigmoid', 'Mish', 'Mish']
actfn2 = ['tanh', 'Sigmoid', 'Sigmoid']

test_days = [3, 3, 15]

for i in range(len(actfn1)):

    act1fn1=actfn1[i]
    act2fn2=actfn2[i]
    #number of days as features
    day_feat = test_days[i]
    #number of days to use as features
    day_targ = 1
    day_targ = day_targ - 1
    
    for j in range(len(stock_names)):

        sn = stock_names[j]
        f = r'G:\My Drive\School\UNCC\P.h.D\Coursework\2023 - SPRING\ITCS 8156 - Machine Learning\Project\archive'
        X_train, X_test, T_train, T_test = market_prepro(f,st,sn,False,splitdata=True, stdzr='minmax')

        # dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train[['Open','Low']], T_train, 4, 1,None)
        # dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test[['Open','Low']], T_test, 4, 1,  None)

        dl_train, ds_train = lstm_timeseries_feat_and_targ(X_train, T_train, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])
        dl_test, ds_test = lstm_timeseries_feat_and_targ(X_test, T_test, day_feat, day_targ, [ 'Year', 'Month' ,'Day_date', 'Day'])

        mdl_stock = BasicLSTM(num_feat=7, num_hiddens=1, num_out=1, lr=0.01, actfn1=act1fn1, actfn2=act2fn2)
        mdl_stock.forward(ds_train[0][0])

        logger = TensorBoardLogger("lightning_logs", name="market")
        trainer = pl.Trainer(max_epochs=10,logger=logger) # with default learning rate, 0.001 (this tiny learning rate makes learning slow)
        trainer.fit(mdl_stock, train_dataloaders=dl_train)
        trainer.test(mdl_stock,dataloaders=dl_test)
        torch.save(mdl_stock, act1fn1+act2fn2+str(day_feat)+sn)

        y_test, t_test = makepred(mdl_stock, ds_test)
        y_train, t_train = makepred(mdl_stock, ds_train)

        train_score = r2_score(t_train, y_train)
        test_score = r2_score(t_test, y_test)

        add_res_sweep = [act1fn1, act2fn2, sn, day_feat, train_score, test_score]

        results_stock_sweep.append(add_res_sweep)


        plt.figure()
        plt.plot(range(len(t_test)),t_test)
        plt.plot(range(len(y_test)),y_test)
        plt.title(act1fn1+act2fn2+' '+str(day_feat)+' '+sn)

In [ ]:
torch.save(results_stock_sweep, 'Stock Sweep Results')